#Restricted Boltzmann Machines (RBM)

首先需要具备的知识：
- 玻尔兹曼机
- 二部图
- sigmoid函数
- Bayes定理
- 蒙塔卡罗方法
- 马尔科夫链
- MCMC

##Energy-Based Models (EBM) 
给予能量的模型就是通过一些属性，计算出一个标量的能量。我们通过学习将能量函数向我们希望的方向改进。例如根据我们的判断希望得到更低的能量。基于能量的模型就是通过能量方程定义一个概率分布模型：
$$p(x)=\frac{e^{-E(x)}}{Z}$$
此时E为能量函数，p为概率模型，我们要让我们的结果计算出的概率最大（可以看出此时能量应该最小），又因为我们要用梯度下降法求解，所以我们可以给p加上负号-p来让-p最小
Z是用来正规化的参数：
$$Z=\sum_x{e^{-E(x)}}$$
一般能量函数可以通过随机梯度下降法优化（：the empirical negative log-likelihood of the training data实验负数log相似度）。

在线性回归中softmax函数是一个例子
$$E(x)= x_{max} - x（1）$$
$$p(x) = \frac{e^{x-x_{max}}}{\sum_{x\in D}{e^{x-x_{max}}}}（2）$$
下面是梯度下降的过程，目的就是让 negativelog-likelihood最小,如下：
$$\iota(\theta,D) = -\frac{1}{N}\sum{\log{p(x,\theta)}}  （3）$$

为了让模型更有表现力，我们引入一些属性，但是这些属性是未知的我们用$h$表示，原本已知的属性还是用x表示：那么P写作：
$$p(x) = \sum_h{P(x,h)}=\sum_h{\frac{e^{-E(x,h)}}{Z}}$$
为了和之前的内容保持一致性，作一下修改：
我们另：$-F(x)$代替$-E(x)$:
$$F(x)=-\log{\sum_h{e^{E(x,h)}}}$$
则：
$$p(x)=\frac{e^{-F(x)}}{Z}$$

$$Z=\sum_x{e^{-F(x)}}$$

#上边给出了概率分布
下面讨论如何让这个含有隐藏属性概率的最大

首先我们的negative log-likelihood gradient如公式（3），这个时候需要通过梯度下降法求解，所以我们计算梯度(尝试推倒一下，很容易得到结果)：
$$-\frac{\partial \log{p(x)}}{\partial \theta} =\frac{\partial F(x)}{\partial \theta}-\sum_x{p(x)\frac{\partial F(x)}{\partial \theta}}$$
可以参考的网址：http://www.cnblogs.com/tornadomeet/archive/2013/03/27/2984725.html

然而这个导数很难求，所以我们用蒙塔卡罗方法（统计模拟方法，使用随机数来解决计算问题）来进行简化。
- 简化第一步：使用期望替代p(X),那么x就是服从p采样规律的
现在的问题就是如何得到期望： Markov Chain Monte Carlo methods 

介绍波尔兹曼机（来源于百度百科）：

玻尔兹曼机：BM是一种对称耦合的随机反馈型二值单元神经网络，由可见层和多个隐层组成，网络节点分为可见单元(visible unit)和隐单元(hidden unit)，用可见单元和隐单元来表达随机网络与随机环境的学习模型，通过权值表达单元之间的相关性

受限玻尔兹曼机：RBM是一种玻尔兹曼机的变体，但限定模型必须为二分图。模型中包含对应输入参数的输入（可见）单元和对应训练结果的隐单元，图中的每条边必须连接一个可见单元和一个隐单元。（与此相对，“无限制”玻尔兹曼机包含隐单元间的边，使之成为递归神经网络。）这一限定使得相比一般玻尔兹曼机更高效的训练算法成为可能，特别是基于梯度的对比分歧（contrastivedivergence）算法

#代码实现


In [4]:
import timeit

try:
    import PIL.Image as Image
except ImportError:
    import Image

import numpy

import theano
import theano.tensor as T
import os

from theano.tensor.shared_randomstreams import RandomStreams

from utils import tile_raster_images
from mlp_lz import load_data

首先引入功能包，下面定义类型

In [7]:
class RBM(object):
    def __init(self,input,n_vsible=784,n_hidden=500,N=None,hbias=None,vBias=None,numpy_rng=None,theano_rng=None):
        #
        self.n_visible
        self.n_hidden
        
        if numpy_rng is None:
            numpy_rng = numpy.random.RandomState(1234)
        #用于共哦共享的随机数
        if theano_rng is None:
            theano_rng = RandomStreams(numpy_rng.randint(2**30))
        
        if W is None:
            initial_W = numpy.assary(
                numpy_rng.uniform(
                    low = -4 * numpy.sqrt(6./(n_hidden + n_visible)),
                    high =4  * numpy.sqrt(6./(n_hidden + n_visible)),
                ),
                dtype = theano.config.floatX
            )
            W = theano.shared(value = initial_W , name = 'W' , brrow = True)
        
        if hbais is None:
            hbias = theano.shared(
                value = numpy.zeros(
                    n_hidden,
                    dtype = theano.config.floatX
                ),
                name = 'hbias',
                borrow = True
            )
        if vbias is None:
            # create shared variable for visible units bias
            vbias = theano.shared(
                value=numpy.zeros(
                    n_visible,
                    dtype=theano.config.floatX
                ),
                name='vbias',
                borrow=True
            )
        #保存输入类型
        self.input = input
        if not input:
            self.input = T.matrix('input')
        
        
        self.W = W
        self.hbias = hbias
        self.vbias = vbias
        self.theano_rng = theano_rng
        # **** WARNING: It is not a good idea to put things in this list
        # other than shared variables created in this function.
        self.params = [self.W, self.hbias, self.vbias]
        # end-snippet-1
        
        def free_energy(self , v_sample):
            wx_b = T.dot(v_sample , self.W) + self.hbias
            vbias_term = T.dot(v_sample,self.vbias)
            hidden_term = T.sum(T.log(1+T.exp(wx_b)),axis = 1)
            return -hidden_term - vbias_term
        
        #定义前向传播函数 #可视层进入隐藏层
        def propup(self,vis):
            pre_sigmoid_activation = T.dot(vis , self.W) + self.hbias
            return [pre_sigmoid_activation , T.nnet.sigomid(pre_sigmoid_activation)]
        #反向传播曾 从隐藏层计算可是曾
        def propdown(self,hid):
            pre_sigmoid_activation =  T.dot(hid , self.W.T) + self.vbias
            return [pre_sigmoid_activation , T.nnet.sigomid(pre_sigmoid_activation)]
        
        #通过输入层计算激活层的状态（根据概率进行确定）
        def sample_h_given_v(self,v0_sample):
            pre_sigmoid_h1 , h1_mean = self.propup(v0_sample)
            h1_sample = self.theano_rng.binomial(size = h1_mean.shape,n=1,p=h1_mean,
                                                dtype=theano.config.floatX)
            return [pre_sigmoid_h1,h1_mean,h1_sample]
        
        def sample_v_given_h(self,h0_sample):
            pre_sigmoid_v1 , v1_mean = self.propdown(h0_sample)
            v1_sample = self.theano_rng.binomial(size = v1_mean.shape,n=1,p=v1_mean,
                                                dtype = theano.config.floatX)
            return [pre_sigmoid_v1 ,v1_mean,v1_sample]
        
        #Gibbs采样 是用来使RBM（实际上就是一个函数）的分布逼近样本的分布？？？
        #对RBM进行采样
        def gibbs_hvh(self,h0_sample):
            pre_sigmoid_v1,v1_mean,v1_sample = self.sample_v_given_h(h0_sample)
            pre_sigmoid_h1,h1_mean,h1_sample = self.sample_h_given_v(v0_sample)
            return [pre_sigmoid_v1,v1_mean,v1_sample,pre_sigmoid_h1,h1_mean,h1_sample]
        
        #改进CD
        def gibbs_vhv(self,h0_sample):
            pre_sigmoid_h1,h1_mean,h1_sample = self.sample_h_given_v(v0_sample)
            pre_sigmoid_v1,v1_mean,v1_sample = self.sample_v_given_h(h0_sample)
            return [pre_sigmoid_h1,h1_mean,h1_sample,pre_sigmoid_v1,v1_mean,v1_sample]
        
        
        def get_cost_updates(self, lr=0.1, persistent=None, k=1):

            # compute positive phase
            pre_sigmoid_ph, ph_mean, ph_sample = self.sample_h_given_v(self.input)

            # decide how to initialize persistent chain:
            # for CD, we use the newly generate hidden sample
            # for PCD, we initialize from the old state of the chain
            if persistent is None:
                chain_start = ph_sample
            else:
                chain_start = persistent
       
            (
                [
                    pre_sigmoid_nvs,
                    nv_means,
                    nv_samples,
                    pre_sigmoid_nhs,
                    nh_means,
                    nh_samples
                ],
                updates
            ) = theano.scan(
                self.gibbs_hvh,
                outputs_info=[None, None, None, None, None, chain_start],
                n_steps=k
            )
          
            # determine gradients on RBM parameters
            # note that we only need the sample at the end of the chain
            chain_end = nv_samples[-1]

            cost = T.mean(self.free_energy(self.input)) - T.mean(
                self.free_energy(chain_end))
            # We must not compute the gradient through the gibbs sampling
            gparams = T.grad(cost, self.params, consider_constant=[chain_end])
            # end-snippet-3 start-snippet-4
            # constructs the update dictionary
            for gparam, param in zip(gparams, self.params):
                # make sure that the learning rate is of the right dtype
                updates[param] = param - gparam * T.cast(
                    lr,
                    dtype=theano.config.floatX
                )
            if persistent:
                # Note that this works only if persistent is a shared variable
                updates[persistent] = nh_samples[-1]
                # pseudo-likelihood is a better proxy for PCD
                monitoring_cost = self.get_pseudo_likelihood_cost(updates)
            else:
                # reconstruction cross-entropy is a better proxy for CD
                monitoring_cost = self.get_reconstruction_cost(updates,
                                                               pre_sigmoid_nvs[-1])

            return monitoring_cost, updates
            # end-snippet-4

            
#参数解释：学习速率，训练迭代次数，数据集，每次训练规模，          
def test_rbm(learning_rate=0.1, training_epochs=15,
             dataset='mnist.pkl.gz', batch_size=20,
             n_chains=20, n_samples=10, output_folder='rbm_plots',
             n_hidden=500):
    
    datasets = load_data(dataset)

    #只需要训练集和测试集
    train_set_x, train_set_y = datasets[0]
    test_set_x, test_set_y = datasets[2]

    n_train_batches = train_set_x.get_value(borrow=True).shape[0] / batch_size
    
    #定位数据位置
    index = T.lscalar()    # index to a [mini]batch
    x = T.matrix('x')  # the data is presented as rasterized images

    #随机数生成器
    rng = numpy.random.RandomState(123)
    theano_rng = RandomStreams(rng.randint(2 ** 30))

    # initialize storage for the persistent chain (state = hidden
    # layer of chain)
    persistent_chain = theano.shared(numpy.zeros((batch_size, n_hidden),
                                                 dtype=theano.config.floatX),
                                     borrow=True)

    # 构造RBM
    rbm = RBM(input=x, n_visible=28 * 28,
              n_hidden=n_hidden, numpy_rng=rng, theano_rng=theano_rng)

    # get the cost and the gradient corresponding to one step of CD-15
    cost, updates = rbm.get_cost_updates(lr=learning_rate,
                                         persistent=persistent_chain, k=15)

    #################################
    #     Training the RBM          #
    #################################
    if not os.path.isdir(output_folder):
        os.makedirs(output_folder)
    os.chdir(output_folder)

    # start-snippet-5
    # it is ok for a theano function to have no output
    # the purpose of train_rbm is solely to update the RBM parameters
    train_rbm = theano.function(
        [index],
        cost,
        updates=updates,
        givens={
            x: train_set_x[index * batch_size: (index + 1) * batch_size]
        },
        name='train_rbm'
    )

    plotting_time = 0.
    start_time = timeit.default_timer()

    # go through training epochs
    for epoch in xrange(training_epochs):

        # go through the training set
        mean_cost = []
        for batch_index in xrange(n_train_batches):
            mean_cost += [train_rbm(batch_index)]

        print 'Training epoch %d, cost is ' % epoch, numpy.mean(mean_cost)

        # Plot filters after each training epoch
        plotting_start = timeit.default_timer()
        # Construct image from the weight matrix
        image = Image.fromarray(
            tile_raster_images(
                X=rbm.W.get_value(borrow=True).T,
                img_shape=(28, 28),
                tile_shape=(10, 10),
                tile_spacing=(1, 1)
            )
        )
        image.save('filters_at_epoch_%i.png' % epoch)
        plotting_stop = timeit.default_timer()
        plotting_time += (plotting_stop - plotting_start)

    end_time = timeit.default_timer()

    pretraining_time = (end_time - start_time) - plotting_time

    print ('Training took %f minutes' % (pretraining_time / 60.))
    # end-snippet-5 start-snippet-6
    #################################
    #     Sampling from the RBM     #
    #################################
    # find out the number of test samples
    number_of_test_samples = test_set_x.get_value(borrow=True).shape[0]

    # pick random test examples, with which to initialize the persistent chain
    test_idx = rng.randint(number_of_test_samples - n_chains)
    persistent_vis_chain = theano.shared(
        numpy.asarray(
            test_set_x.get_value(borrow=True)[test_idx:test_idx + n_chains],
            dtype=theano.config.floatX
        )
    )
    # end-snippet-6 start-snippet-7
    plot_every = 1000
    # define one step of Gibbs sampling (mf = mean-field) define a
    # function that does `plot_every` steps before returning the
    # sample for plotting
    (
        [
            presig_hids,
            hid_mfs,
            hid_samples,
            presig_vis,
            vis_mfs,
            vis_samples
        ],
        updates
    ) = theano.scan(
        rbm.gibbs_vhv,
        outputs_info=[None, None, None, None, None, persistent_vis_chain],
        n_steps=plot_every
    )

    # add to updates the shared variable that takes care of our persistent
    # chain :.
    updates.update({persistent_vis_chain: vis_samples[-1]})
    # construct the function that implements our persistent chain.
    # we generate the "mean field" activations for plotting and the actual
    # samples for reinitializing the state of our persistent chain
    sample_fn = theano.function(
        [],
        [
            vis_mfs[-1],
            vis_samples[-1]
        ],
        updates=updates,
        name='sample_fn'
    )

    # create a space to store the image for plotting ( we need to leave
    # room for the tile_spacing as well)
    image_data = numpy.zeros(
        (29 * n_samples + 1, 29 * n_chains - 1),
        dtype='uint8'
    )
    for idx in xrange(n_samples):
        # generate `plot_every` intermediate samples that we discard,
        # because successive samples in the chain are too correlated
        vis_mf, vis_sample = sample_fn()
        print ' ... plotting sample ', idx
        image_data[29 * idx:29 * idx + 28, :] = tile_raster_images(
            X=vis_mf,
            img_shape=(28, 28),
            tile_shape=(1, n_chains),
            tile_spacing=(1, 1)
        )

    # construct image
    image = Image.fromarray(image_data)
    image.save('samples.png')
    # end-snippet-7
    os.chdir('../')
